<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/interactive_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive Visualization

In [ ]:
# Upgrade Matplotlib
!pip install matplotlib --upgrade
!pip install plotly --upgrade

In [ ]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
#setting plot size
plt.rcParams["figure.figsize"] = (7.0,4.0)

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [ ]:
# Storing loaded data from excel to a pandas dataframe
import io
df1 = pd.read_excel(io.BytesIO(uploaded1['Script_202210261607.xlsx'])) #2022-10-14-script.xlsx

In [ ]:
# Chainging columns to lower case
df1.rename(columns={i:i.lower() for i in df1.columns}, inplace=True)

In [ ]:
# Creating time categories
df1 = df1.assign(year = df1['order_date'].dt.year,
                 quarter = df1['order_date'].dt.quarter,
                 month = df1['order_date'].dt.month,
                 month_name = df1['order_date'].dt.month_name(),
                 weeknum_order = df1['order_date'].dt.isocalendar().week,
                 weeknum_activation = df1['activation_date'].dt.isocalendar().week,
                 weeknum_cancellation = df1['cancellation_date'].dt.isocalendar().week,
                 day = df1['order_date'].dt.day_name()
                 )

In [ ]:
# Checking the dataframe info
df1.info()

In [ ]:
df1.head(5)

In [ ]:
# Splitting columns to lists for interactive visualization
date_col = df1.select_dtypes(include=['datetime64[ns]']).columns.tolist()
dim_col = df1.select_dtypes(include=['object']).columns.tolist()
eve_col = [i for i in dim_col if ('event') in i]
cat_col = [i for i in dim_col if ('category') in i]
num_col = df1.select_dtypes(include=['float64', 'int64']).columns.tolist()

In [ ]:
cat_col

## Visualization products

In [ ]:
# Exploring numeric columns by Category
@interact(Period=date_col, Dimension=df1[dim_col[0]].unique(), Category=cat_col, Measure=num_col)
def visualize_category(Period, Dimension, Category, Measure):
  df = data=df1[(df1[dim_col[0]]==Dimension)].copy()
  sns.lineplot(x=Period, y=Measure, data=df, hue=Category)
  #plt.title(f'Product_type: {Dimension}; Measure: {Measure}; Category: {Category}')
  plt.title(f'Product_type: {Dimension}; Measure: {Measure}')
  plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()

In [ ]:
# Exploring numeric columns
@interact(Period=date_col, Dimension=df1[dim_col[0]].unique(), Category=cat_col, Measure=num_col)
def visualize_statistics(Period, Dimension, Category):
  df = data=df1[(df1[dim_col[0]]==Dimension)&(df1[cat_col[0]]=='Has_Usage')].copy()
  sns.lineplot(x=Period, y=num_col[2], data=df, color='red')
  #sns.lineplot(x=Period, y=num_col[3], data=df)
  #sns.lineplot(x=Period, y=num_col[4], data=df)
  sns.lineplot(x=Period, y=num_col[5], data=df, color='black')
  plt.title(f'Product_type: {Dimension}; Category: Has_Usage')
  plt.ylabel('MB', fontsize=12)
  plt.legend(labels=[num_col[2], num_col[5]], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()

In [ ]:
# Running total of numeric columns by Category
@interact(Period=date_col, Dimension=df1[dim_col[0]].unique(), Category=cat_col, Measure=num_col)
def visualize_category(Period, Dimension, Category, Measure):
  df = data=df1[(df1[dim_col[0]]==Dimension)].copy()
  df = df.assign()
  sns.lineplot(x=Period, y=Measure, data=df, hue=Category)
  #plt.title(f'Product_type: {Dimension}; Measure: {Measure}; Category: {Category}')
  plt.title(f'Product_type: {Dimension}; Measure: {Measure}')
  plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()

## Visualization Binding

In [ ]:
# Exploring dimension columns by Category
@interact(Period=['2022-09-30'], Event=df1[eve_col[0]].unique(), Binding=['Yes', 'No'], Category=df1[cat_col[0]].unique(), Dimension=dim_col, Percentage=[True, False])
def explore_value_counts(Period, Category, Event, Binding, Dimension, Percentage):
  df = df1[(df1[date_col[0]]==Period)&(df1[eve_col[0]]==Event)&(df1['has_contract_commitment_category']==Binding)&(df1[cat_col[0]]==Category)].copy().reset_index(drop=True)
  #return df
  return df.value_counts(subset=[Dimension], normalize=Percentage, dropna=False).head()

In [ ]:
# Exploring dimension columns by Category
@interact(Period=['2022-09-30'], Event=df1[eve_col[0]].unique(), Binding=['Yes', 'No'], Category=df1[cat_col[0]].unique(), Measure=num_col)
def visualize_bar_plot(Period, Category, Event, Measure):
  df = df1[(df1[date_col[0]]==Period)&(df1[cat_col[0]]==Category)&(df1[eve_col[0]]==Event)].copy().reset_index(drop=True) #&(df1[eve_col[0]]==Event)
  #return df
  fig_x, axes = plt.subplots(1, 1, figsize=(8.5, 5.5))
  axes.set_title(f'{Category} - {Event} - {Measure}')
  #sns.boxplot(data=df, y=Measure, x='has_contract_commitment_category', ax=axes) #multiple="stack",
  #sns.histplot(data=df, x=Measure, hue='has_contract_commitment_category', alpha=0.5, ax=axes)  #multiple="stack",
  #sns.violinplot(data=df, x=Measure, y='has_contract_commitment_category', hue='event_type', kind="violin", bw=.25, cut=0, split=True, ax=axes)
  sns.barplot(data=df, x=Measure, y='has_contract_commitment_category', order=['Yes', 'No'], ax=axes)
  #plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  #axes.set_yticklabels(['Yes', 'No'])
  plt.show()